<a href="https://colab.research.google.com/github/sbihaiko/magicGUI/blob/main/colab_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI

## Setup

### Install

In [14]:
import os
import getpass
from google.colab import drive
from IPython.display import clear_output

global OPENAI_KEY

#descomente as alinhas abaixo para resetar o token 
os.environ.pop("OPENAI_API_KEY")
OPENAI_KEY = "chatcmpl-8IJG2XYyzsRsydBviKjjilqrWU5Ve"

try:
    if not OPENAI_KEY:
        OPENAI_KEY = getpass.getpass("OpenAI Token:")
except:
    OPENAI_KEY = getpass.getpass("OpenAI Token:")

os.environ["OPENAI_API_KEY"] = OPENAI_KEY

clear_output()
print("\nMontando GDrive e Diretórios do Projeto...\n")

drive.mount('/content/drive')

global PROJECT_NAME
PROJECT_NAME = "colab.ai"

path = "/content/drive/MyDrive/{proj}".format(proj=PROJECT_NAME)
if not os.path.exists(path):
  os.makedirs(path)

clear_output()
print("\nOpenAI Token Armazenado e GDrive Conectado!\n")   



OpenAI Token Armazenado e GDrive Conectado!



In [15]:
import sys
import shutil

print("\nInstalando o GitHub CLI...\n")

!type -p curl >/dev/null || (sudo apt update && sudo apt install curl -y)
!curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg \
&& sudo chmod go+r /usr/share/keyrings/githubcli-archive-keyring.gpg \
&& echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null \
&& sudo apt update \
&& sudo apt install gh -y
!sudo apt install gh -y

clear_output()
print("\nClonando o Repositório do JaiBot...\n")

git_path = "/content/git"
if os.path.exists(git_path):
  shutil.rmtree(git_path)

os.makedirs(git_path)

token = "/content/drive/MyDrive/{proj}/ssh/git-token".format(proj=PROJECT_NAME)
if not os.path.exists(token):
  os.makedirs("/content/drive/MyDrive/{proj}/ssh".format(proj=PROJECT_NAME))

!cd /content/git \
&& gh auth login --with-token < /content/drive/MyDrive/colab.ai/ssh/git-token \
&& gh repo clone renanwolf/jai-bot-server

clear_output()
print("\nInstalando o JaiBot...\n")

sys.path.append('/content/git/jai-bot-server/')
!cd /content/git \
&& pip install -e /content/git/jai-bot-server/

clear_output()
print("\nJaiBot Instalado!\n")



JaiBot Instalado!



In [16]:
# logger manager hack
import logging
from src.commons import async_utils, logger_manager

logger_manager.setupLogger(logging.INFO)
logger = logger_manager.getLogger("colab")
print()
logger.info("loggers inicializados")
print()
logger_manager.setupLogger(logging.ERROR)
logger.info("loggers inicializados")



[INFO] [logger_manager] [info @27] -> loggers inicializados



In [17]:
clear_output()
print("\nInstalando bibliotecas langchain...\n")
!pip install openai langchain chromadb

clear_output()
print("\nInstalando bibliotecas de crawling...\n")
!pip install beautifulsoup4 requests nltk goose3 PyMuPDF gdown pypdf
!pip install transformers PyPDF2

clear_output()
print("\nInstalação Python completa!\n")



Instalação Python completa!



### Imports e Globals

In [18]:

import re
import asyncio
import json
import gdown
import requests
import shutil
import fcntl
import pandas
import panel
import logging
import hashlib
import nest_asyncio
import ipywidgets
import threading
import time
import fitz
import PyPDF2
import ipywidgets as widgets

from goose3 import Goose
from goose3.configuration import Configuration
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from io import BytesIO
from google.colab import files
from panel.widgets import Tabulator, Button
from panel.io.notebook import load_notebook
from IPython.display import display, Javascript

from langchain import embeddings
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate  
from langchain.document_loaders import TextLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

from src.apis.documents_wizard.url_extractor import UrlExtractor
from src.apis.documents_wizard.ai_summarizer import AISummarizer
from src.processor_questions_docs_answering.ai_questions_docs_answering import AIQuestionsDocsAnswering
from src.commons.models import utils as model_utils
from src.commons.documents.semantic_manager import DocsSemanticManager
from src.commons.documents.models import DocumentRef, DocumentData, Document
from transformers import T5Tokenizer, T5ForConditionalGeneration

nest_asyncio.apply()

# Seta Constantes Globais

global DB_PATH
global STAGE_PATH

global DARK_GRAY
global LIGHT_GRAY
global BLACK

DB_PATH = "/content/drive/MyDrive/{proj}/{kd}/{stage}/chromaDB"
LIGHT_GRAY = "#FFFFFF"
DARK_GRAY = "#888888"
BLACK = "#000000"

clear_output()
print()
print(f"DB_PATH: \t\t{DB_PATH}")
print(f"PROJECT_NAME: \t\t{PROJECT_NAME}")
print(f"LIGHT_GRAY: \t\t{LIGHT_GRAY}")
print(f"DARK_GRAY: \t\t{DARK_GRAY}")
print(f"BLACK: \t\t\t{BLACK}")

print("\nVariáveis Globais Inicializadas!\n")



DB_PATH: 		/content/drive/MyDrive/{proj}/{kd}/{stage}/chromaDB
PROJECT_NAME: 		colab.ai
LIGHT_GRAY: 		#FFFFFF
DARK_GRAY: 		#888888
BLACK: 			#000000

Variáveis Globais Inicializadas!



### JaiBot Facade

In [19]:
os.environ["JAI_EFS_MOUNT"] = "fake_efs_mount_path"
os.environ["JAI_EFS_GIT_ROOT"] = "/content/git/jai-bot-server"
os.environ["JAI_DOCS_DYNAMO_DOCUMENTS_REF"] = "fake_dynamo_dosc_ref"

def extract(url, timeout=10.0):
    async def aextract(url, timeout):
        logging.getLogger('src.apis.documents_wizard.url_extractor').disabled = True
        extractor = UrlExtractor()
        urlInfo = await extractor.aextract(url, timeout)
        return urlInfo.toJsonObject()
        
    return asyncio.run(aextract(url, timeout))

def summarize(content, title = None):
    async def asummarize(content, title = None):
        summarizer = AISummarizer(timeout = 30.0)
        summarizer.setContentSummarization(content, title)
        return await summarizer.asummarize()

    return asyncio.run(asummarize(content, title))

def buildVectorStore(campaingn_name):
    async def aimpl():
        campaigns = endpoint_getCampaigns()
        confs = campaigns[campaingn_name]["confs"]
        os.environ["JAI_DOCS_SEMANTIC_CHUNK_SIZE"] = str(confs["chunk_size"])
        os.environ["JAI_DOCS_SEMANTIC_CHUNK_OVERLAP"] = str(confs["chunk_overlap"])
        semanticManager = DocsSemanticManager()
        stages = campaigns[campaingn_name]["stages"]
        for stageName, stage in stages.items():
            dbPath = DB_PATH.format(kd=campaingn_name, stage=stageName, proj=PROJECT_NAME)
            if os.path.exists(dbPath):
                shutil.rmtree(dbPath)
            os.makedirs(dbPath)
            print(f'\n{campaingn_name}/{stageName}: {confs["chunk_size"]} | {confs["chunk_overlap"]}')

            for doc in stage["docs"]:
                url = doc["file"]
                md5 = hashlib.md5()
                md5.update(url.encode())
                rawContent = doc.get("content", "")
                imprContent = doc.get("improved", "")
                docRef, chunkRaw, chunkImpr, exInfo = await semanticManager._agenerateChunksFromDocumentData(
                    DocumentRef("accountColab1", campaingn_name, md5.hexdigest()),
                    DocumentData(content = rawContent, improvedContent = imprContent),
                    None
                )
                if exInfo:
                    raise exInfo.recreate()
                docRef, exInfo = (await semanticManager._insertChunksIntoVectoreStoreOfAccountCampaign(
                    dbPath, [(docRef, chunkRaw, chunkImpr, exInfo)]
                ))[0]
                if exInfo:
                    raise exInfo.recreate()
    return asyncio.run(aimpl())

def answer_me(campaingn_name, stage_name, use_prompt, query):
    
    print(f"\n{campaingn_name} | {stage_name}")

    CHAT_TYPE = os.environ.get('CHAT_TYPE')
    CHAIN_TYPE =  os.environ.get('CHAIN_TYPE')
    SEARCH_TYPE = os.environ.get('SEARCH_TYPE')
    RELEVANT_CHUNKS = int(os.environ.get('RELEVANT_CHUNKS'))
    #print(f"\n{campaingn_name}/{stage_name}: {CHAT_TYPE} | {CHAIN_TYPE} | {SEARCH_TYPE} | {RELEVANT_CHUNKS}")

    # <JAI_EFS_MOUNT>/custom_prompts_for_account_campaign/accountId/campaignId/prompt_config_questions_answering.yaml
    async def aimpl():
        dbPath = DB_PATH.format(kd=campaingn_name, stage=stage_name, proj=PROJECT_NAME)
        semanticManager = DocsSemanticManager()
        async with semanticManager._avectorStoreContext(dbPath, fcntl.LOCK_SH, lockTimeout=5.0, createVectorStore=False, persist=False) as vectorStore:
            aiAnswering = AIQuestionsDocsAnswering.fromPromptConfigYamlFiles(accountId = model_utils.JOINZAP_ACCOUNT_ID, campaignId = campaingn_name)
            if not use_prompt:
                aiAnswering._promptConfig.template = ""
            aiAnswering._promptConfig.temperature = 0.7
            aiAnswering._promptConfig.chain.chainType = os.environ["CHAIN_TYPE"]
            aiAnswering._promptConfig.retriever.searchType = os.environ["SEARCH_TYPE"]
            aiAnswering._promptConfig.retriever.k = int(os.environ["RELEVANT_CHUNKS"])
            return await aiAnswering.aexecute(vectorStore, query, verbose = False)
    return asyncio.run(aimpl())


print("\nJaiBot Facade Carregada!\n")
#answer_me("Adriano", "PDFs", False, "[VOCÊ] oi")


JaiBot Facade Carregada!



### Common API

In [20]:
# Carrega Common API
def storeFile(data, filePath):
  with open(filePath, "w") as f:
      json.dump(js, f, ensure_ascii=False, indent=2)

def storeJSon(js, filePath):
  with open(filePath, "w") as f:
      json.dump(js, f, ensure_ascii=False, indent=2)


def restoreJSon(filePath):
  with open(filePath, 'r') as f:
      return json.load(f)

def loadCampaignsDataframe(campaign, stage):
    campaigns = endpoint_getCampaigns()
    camps = []
    stages = []
    files = []

    for key, c in campaigns.items():
        if key != campaign: continue
        for key2, s in c["stages"].items():
            if key2 != stage: continue
            for doc in s['docs']:
                camps.append(campaign)
                stages.append(key2)
                files.append(doc["file"])

    return pandas.DataFrame({
        "Campanha": camps,
        "Estágio": stages,
        "Arquivo": files
    })


def printJSon(js):
  print(json.dumps(js, ensure_ascii=False, indent=2))


def getDB(campaign_name, stage):

    campaigns = endpoint_getCampaigns()
    embedding_type = campaigns[campaign_name]["confs"]["embeddings"]

    if embedding_type == "HuggingFaceEmbeddings": 
        embedding = HuggingFaceEmbeddings()
    else: 
        embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

    dbPath = DB_PATH.format(kd=campaign_name, stage= stage, proj=PROJECT_NAME)
    return Chroma(persist_directory = dbPath, embedding_function = embedding)


def doChat(campaign, stage):

    last_campaign =  os.environ.get('ACTIVE_CAMPAIGN')
    last_stage = os.environ.get('ACTIVE_STAGE')
    try:
        while True:

            ACTIVE_CAMPAIGN =  os.environ.get('ACTIVE_CAMPAIGN')
            ACTIVE_STAGE = os.environ.get('ACTIVE_STAGE')
            if ACTIVE_CAMPAIGN!=last_campaign or ACTIVE_STAGE!=last_stage:
                clear_output(wait=True)


            CHAT_TYPE = os.environ.get('CHAT_TYPE')
            user_input = input("\nEntre sua pergunta: ")

            query=f"[VOCÊ] {user_input}"

            log.append(query)

            answer = answer_me(campaign, stage, (CHAT_TYPE != "QNA"), query)
            clear_output(wait=True)

            log.append(f"[AI] {answer}")
            log.append(' ')

            print()
            for string in log:
                words = string.split()
                result = ""
                for word in words:
                    if len(result + word) + 1 > 100:
                        print(result)
                        result = word
                    elif result == "":
                        result = word
                    else:
                        result += " " + word
                print(result)

    except KeyboardInterrupt:
        print("Fim do chat.")
    except Exception as e:
        raise

print("\nCommon API Carregada!\n")

def do_chat():
    campaign =  os.environ.get('ACTIVE_CAMPAIGN')
    stage = os.environ.get('ACTIVE_STAGE')
    print(f"{campaign} | {stage}")

    CHAT_TYPE = os.environ.get('CHAT_TYPE')
    CHAIN_TYPE =  os.environ.get('CHAIN_TYPE')
    SEARCH_TYPE = os.environ.get('SEARCH_TYPE')
    RELEVANT_CHUNKS = int(os.environ.get('RELEVANT_CHUNKS'))

    print(f"{CHAT_TYPE} | {CHAIN_TYPE} | {SEARCH_TYPE} | {RELEVANT_CHUNKS} ")

    doChat(campaign, stage)

def extract_from_pdf(url):

    tmp = "/content/drive/MyDrive/{proj}/tmp.pdf".format(proj=PROJECT_NAME)
    if url.startswith("https://"):
        try: 
            url_parts = url.split('/')
            file_id = url_parts[url_parts.index('d') + 1]
            download_url = f'https://drive.google.com/uc?id={file_id}' 
            gdown.download(download_url, tmp, quiet=False)

        except:
            response = requests.get(url)
            pdf_content = BytesIO(response.content)
            with open(tmp, 'wb') as f:
                f.write(pdf_content.getbuffer())
    else: 
          tmp=url

    doc = fitz.open(tmp)
    text = ""

    for page in doc:
        text += page.get_text("text")

    return text

def summarize(text):
    # Carrega o modelo e tokenizer
    model_name = 't5-3b'
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Define os parâmetros do modelo para a geração do resumo
    max_length = 100
    min_length = 10
    num_beams = 4
    length_penalty = 2.0

    # Codifica o texto de entrada
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt")

    # Gera o resumo usando o modelo
    summary_ids = model.generate(input_ids, 
                                 max_length=max_length, 
                                 min_length=min_length, 
                                 num_beams=num_beams, 
                                 length_penalty=length_penalty)
    
    # Decodifica o resumo gerado pelo modelo
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary


[ERROR] [base_events] [default_exception_handler @1758] -> Future exception was never retrieved
future: <Future finished exception=TypeError("download() got an unexpected keyword argument 'format'")>
Traceback (most recent call last):
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 128, in _extractFromHtmlWithGDownAndSoup
    filePath = self._downloadGDriveFileToTemp(gFileId, "txt")
  File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 94, in _downloadGDriveFileToTemp
    path = gdown.download(id = id, output = path, quiet = False, format = exportFormat, use_cookies = False)
TypeError: download() got an unexpected keyword argument 'format'
[ERROR] [base_events] [default_exception_handler @1758] -> Future exception was never retrieved
future: <Future finished exception=TypeError("download() got an

In [21]:
# Carrega Endpoints
def endpoint_addDocument(campaign, stage, filePath):
    campaigns = endpoint_getCampaigns()
    if not campaign in campaigns:
        campaigns[campaign] = {"stages":{}}

    if not stage in campaigns[campaign]["stages"]:
        campaigns[campaign]["stages"][stage] = {"docs": []}

    tp =  "pdf" if filePath.endswith(".pdf") else "???"

    campaigns[campaign]["stages"][stage]["docs"].append({"file": filePath, "type": tp})
    endpoint_setCampaigns(campaigns)


def endpoint_delDocument(campaign, stage, filePath):
    campaigns = endpoint_getCampaigns()
    files = campaigns[campaign]["stages"][stage]["docs"]   
    for f in files:
        if f["file"] != filePath: continue;
        files.remove(f)
        break

    endpoint_setCampaigns(campaigns)


def endpoint_setCampaigns(campaigns):   
    pathCampaign = "/content/drive/MyDrive/{proj}/campaigns.json".format(proj=PROJECT_NAME)
    for key, campaign in campaigns.items():
        if not "confs" in campaign:
            campaign["confs"] = {
                "embeddings":"OpenAIEmbeddings",
                "chunk_overlap": 80,
                "chunk_size":750
            }
    return storeJSon(campaigns, pathCampaign)


def endpoint_getCampaigns():   
    pathCampaign = "/content/drive/MyDrive/{proj}/campaigns.json".format(proj=PROJECT_NAME)
    if not os.path.exists(pathCampaign):
        storeJSon([], pathCampaign)
    return restoreJSon(pathCampaign)


def endpoint_addStage(campaign_name, stage):
    campaigns = endpoint_getCampaigns()
    if not campaign_name in campaigns:
        campaigns[campaign_name] = {"stages":{}}

    if not stage in campaigns[campaign_name]["stages"]:
        campaigns[campaign_name]["stages"][stage] = {"docs": []}

    endpoint_setCampaigns(campaigns)


def endpoint_qNa(campaign, stage, query):

    CHAIN_TYPE =  os.environ.get('CHAIN_TYPE')
    SEARCH_TYPE = os.environ.get('SEARCH_TYPE')
    RELEVANT_CHUNKS = int(os.environ.get('RELEVANT_CHUNKS'))

    db = getDB(campaign, stage)
    retriever = db.as_retriever(search_type=SEARCH_TYPE, search_kwargs={"k": RELEVANT_CHUNKS})
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), 
        chain_type=CHAIN_TYPE, 
        retriever=retriever, 
        return_source_documents=True
    )
    
    result = qa({"query": query})
    return result


def endpoint_buildDB(campaign_name):

    campaigns = endpoint_getCampaigns()
    campaign = campaigns[campaign_name]  
    chunk_size = campaign["confs"]["chunk_size"]
    chunk_overlap = campaign["confs"]["chunk_overlap"]
    embedding_type = campaign["confs"]["embeddings"]

    log = []
    def loadDoc(node, chunk_size, chunk_overlap):
        splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        log.append(f" * {node['file']}")   
        if node["file"].startswith("http"):
            if "content" not in node:
                log.append(f"\n{node['file']}")
                result = extract(node["file"])
                node["content"] = result["content"]         
                #node["type"] = result["type"]

            docs = splitter.create_documents([node["content"]]) 
            return splitter.split_documents(docs) 

        loader = PyPDFLoader(node["file"])
        pdf = loader.load()
        return splitter.split_documents(pdf)


    if embedding_type == "HuggingFaceEmbeddings": 
        embedding = HuggingFaceEmbeddings()
    else: 
        embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)


    for key, stage in (campaign["stages"]).items():

        log.append(f"\n{campaign_name} - {key}")      
        dbPath = DB_PATH.format(kd=campaign_name, stage= key, proj=PROJECT_NAME)
        if os.path.exists(dbPath):
            shutil.rmtree(dbPath)
        os.makedirs(dbPath)

        files = stage["docs"]
        chuncks = []

        for node in files: 
            chuncks.extend(loadDoc(node, chunk_size, chunk_overlap))
            
        vectordb = Chroma.from_documents(
            documents=chuncks, 
            embedding=embedding, 
            persist_directory=dbPath
        ).persist()

    endpoint_setCampaigns(campaigns)
    
    clear_output()
    print()
    print(campaign_name)
    printJSon(campaign["confs"])
    for line in log: print(line)


print("\nTodos os EndPoints Carregados!\n")



Todos os EndPoints Carregados!



### Files

In [22]:
# Reset Arquivo de Campanhas
storeJSon({}, "/content/drive/MyDrive/{proj}/campaigns.json".format(proj=PROJECT_NAME))

# Arquivo 72 horas
# endpoint_addDocument("EchoBike", "72 horas", "https://docs.google.com/document/d/1Be3DymbVZaLDC_7rOfTPYtKBpJYn8u8IhN2FqoN5QQc/edit?usp=sharing")
# endpoint_addDocument("EchoBike", "72 horas", "https://docs.google.com/document/d/1Xt1kPHBNZFeEQTiBNQ0ofjmqdQgNc9FQ0zgct_nwZf0/edit?usp=share_link")

# # Aquivos 48 horas
# endpoint_addDocument("EchoBike", "48 horas", "https://docs.google.com/document/d/1Be3DymbVZaLDC_7rOfTPYtKBpJYn8u8IhN2FqoN5QQc/edit?usp=sharing")
# endpoint_addDocument("EchoBike", "48 horas", "https://docs.google.com/document/d/1RupWDWfzNpOCQnMjkGrEPP3Z26SakxjEiu4vmnmfilA/edit?usp=share_link")

# # Aquivos 24 Horas
# endpoint_addDocument("EchoBike", "24 horas", "https://docs.google.com/document/d/1Be3DymbVZaLDC_7rOfTPYtKBpJYn8u8IhN2FqoN5QQc/edit?usp=sharing")
# endpoint_addDocument("EchoBike", "24 horas", "https://docs.google.com/document/d/1sW_dNI6xLTAK8iPeHk0zgoEdXzwFi_u4MSpv_4lOHp4/edit?usp=share_link")

# # Aquivos 10 Horas
# endpoint_addDocument("EchoBike", "10 horas", "https://docs.google.com/document/d/1Be3DymbVZaLDC_7rOfTPYtKBpJYn8u8IhN2FqoN5QQc/edit?usp=sharing")
# endpoint_addDocument("EchoBike", "10 horas", "https://docs.google.com/document/d/1Ayduon4IOSpHncqj4e0v_Gv0fPJgPqHlnq_l8yeubkI/edit?usp=share_link")

# # Aquivos 02 Horas
# endpoint_addDocument("EchoBike", "2 horas", "https://docs.google.com/document/d/1Be3DymbVZaLDC_7rOfTPYtKBpJYn8u8IhN2FqoN5QQc/edit?usp=sharing")
# endpoint_addDocument("EchoBike", "2 horas", "https://docs.google.com/document/d/1yyWdPwCoM0jrP900qzORqQ5Nm6TwSHrXx80LFSiiSkA/edit?usp=share_link")

# # Arquivo 72 horas
# endpoint_addDocument("Perito5k", "72 horas", "https://docs.google.com/document/d/1rLPtHQL8FjERi1-5WAl4rNLXpuIR2E3RJr9J9HSlTpE/edit?usp=share_link")

# # Aquivos 48 horas
# endpoint_addDocument("Perito5k", "48 horas", "https://docs.google.com/document/d/1pLWZwhboJ8z1wxGghhDCJdAFb4WmfElQYFyNwKmtZMQ/edit?usp=share_link")

# # Aquivos 24 Horas
# endpoint_addDocument("Perito5k", "24 horas", "https://docs.google.com/document/d/1d-tE8gUEW6dw7IkeflgsSPGVlqHchl0NECNF8OaQTTs/edit?usp=share_link")

# # Aquivos 10 Horas
# endpoint_addDocument("Perito5k", "10 horas", "https://docs.google.com/document/d/1e22N1W4KxxLoRhjQq2ial1cpGAEadcO6nToCvtZsSaE/edit?usp=share_link")

# # Aquivos 02 Horas
# endpoint_addDocument("Perito5k", "2 horas", "https://docs.google.com/document/d/18DO9-i6oaFeWmPEawzJC4iJjXGer94oFr8cXNTPhOeA/edit?usp=share_link")

# # Arquivos Catavento
endpoint_addDocument("Catavento", "RH", "https://drive.google.com/file/d/1DVXL4jqkcW_pVE-GZFZzS0ab39Iz1nLN/view?usp=share_link")

# Arquivos Adriano
#endpoint_addDocument("Adriano", "PDFs", "/content/drive/MyDrive/colab.ai/Adriano/PDFs/AutorizacaoVenda-Adriano.pdf")

clear_output()
log = []
log.append("Scraping...")
campaigns = endpoint_getCampaigns()

for key, c in campaigns.items():
    for key2, s in c["stages"].items():
        log.append(f"\n{key} | {key2}")
        for doc in s['docs']:
            url = doc["file"]

            try:
                urlInfoObj = extract(url)
                doc["content"] = urlInfoObj["content"]

            except:
                doc["content"] = extract_from_pdf(url)  

            log.append(f" * {url}")
            clear_output(wait=True)
            for line in log: print(line)

            txt = doc["content"]
            # doc["improved"] = summary(txt)

endpoint_setCampaigns(campaigns)
campaigns = endpoint_getCampaigns()

clear_output(wait=True)
for line in log: print(line)

print()
#printJSon(campaigns)
print(campaigns)


Scraping...

Catavento | RH
 * https://drive.google.com/file/d/1DVXL4jqkcW_pVE-GZFZzS0ab39Iz1nLN/view?usp=share_link

{'Catavento': {'stages': {'RH': {'docs': [{'file': 'https://drive.google.com/file/d/1DVXL4jqkcW_pVE-GZFZzS0ab39Iz1nLN/view?usp=share_link', 'type': '???', 'content': '°-> \nD \n• 3/07-0 \\\\\\ \nP\'4010 -59\nManual de \nRecursos Humanos \nPlano de Cargos e Salários \nCódigo de Ética \nOrganização Soda \nCatavento C \ne Educact \n‘L-oCatavento \n01% cultural e educacional \norganização social de ,..ultura \nPress \nde YminiStrÇao \n1. \nc. \n, \n.0.- an \n,( \ne• \nanaZ44, \nÍNDICE \no \nd ot.\n51 \n-0 \n471) \n.4)V \n\'s2(96.14(ZV \nDISPOSIÇÕES GERAIS \n3 \n2. \nAPLICAÇÃO \n3 \n3. \nADMINISTRAÇÃO DE RECURSOS HUMANOS \n3 \n4. \nRECRUTAMENTO E SELEÇÃO DE COLABORADORES \n4 \n4.1. \nRECRUTAMENTO INTERNO \n4 \n4.2. \nRECRUTAMENTO EXTERNO \n4 \n5. \nADMISSÃO E DEMISSÃO \n7 \n5.1. \nADMISSÃO \n7 \n5.2. \nDEMISSÃO \n7 \n6. \nTREINAMENTO E DESENVOLVIMENTO \n8 \n7. \nBENEFÍCIOS \

### GUI Code

In [23]:
# GUI Adiciona e Remove PDFs
def gui_CampaignFiles(refresh_callback):

    panel.extension('tabulator')
    campaigns = endpoint_getCampaigns()
    fileInput = panel.widgets.FileInput(value=None, margin=10)
    txtLink = panel.widgets.TextInput(width=750)
    txtLink.message="0/0"
    margin = 20
    tabs = None

    def new_table(campaign, stage):
        delete_col = {
            'formatter': 'html', 'title': '', 'field': '', 'headerSort': False, 
            'cellClick': 'function(e, cell) {e.stopPropagation(); cell.getRow().delete();}'
        }
        img = "https://cdn3.iconfinder.com/data/icons/linecons-free-vector-icons-pack/32/trash-16.png"
        tag = {'delete': f"<img src='{img}' style='width:16px;height:18px;;margin-top:2px;'/>"}

        df = loadCampaignsDataframe(campaign, stage)
        tb = Tabulator(df, width=900, show_index=False, selectable=True, disabled=True, buttons=tag)
        
        tb.hidden_columns=["Campanha","Estágio"]
        tb.widths= {"Arquivo":858}

        def delete_row(event):
            if str(event.column) != "delete": return
            c = df.at[0,"Campanha"]
            s = df.at[0,"Estágio"]
            path = df.at[0,"Arquivo"]
            print(c)
            print(s)            
            print(path)
            endpoint_delDocument(c, s, path)

            try:
              lista = fileInput.message.split("/")
              layout[0] = load_tabs(int(lista[0]), int(lista[1]))  
            except:
              layout[0] = load_tabs(0, 0)
                           
        tb.on_click(delete_row)
        return tb


    def load_tabs(i, j):

        def add_url(event):
            print(event.new)

        tabs = panel.Tabs(margin=margin, background=LIGHT_GRAY)
        def on_tab_change(event):
            active_campaign = tabs.objects[event.new].name
            active_stage = tabs.objects[event.new][0].name
            fileInput.name = f"{active_campaign}/{active_stage}"
            fileInput.message = f"{event.new}/0"           
            #print(fileInput.name)

        def on_tab2_change(event):
            active_campaign = tabs.objects[tabs.active].name
            tabs2 = event.obj
            active_stage = tabs2.objects[event.new].name
            fileInput.name = f"{active_campaign}/{active_stage}"
            fileInput.message = f"{tabs.active}/{event.new}"
            #print(fileInput.name)

        tabs.param.watch(on_tab_change, "active")

        #btn_add_link = panel.widgets.Button(name='+', width=40)
        #btn_add_link.on_click(add_url)

        for key, c in campaigns.items():
            tabs2 = panel.Tabs(margin=margin, name=key)
            tabs2.param.watch(on_tab2_change, "active")
            tabs.append((key, tabs2))
            for key2, s in c["stages"].items():        
                tb = new_table(key, key2)
                tab = panel.Column(
                            panel.Column(tb),
                            panel.Column(
                                panel.Row("Adicionar Arquivo:", fileInput),
                                #panel.Row("Adicionar URL:", txtLink, btn_add_link),
                                background=LIGHT_GRAY, 
                                width=930
                            ),  background=LIGHT_GRAY

                        , name=key2)
                tabs2.append((key2, tab))

        tabs.active = i
        tabs2 = tabs.objects[tabs.active]
        tabs2.active = j
        return tabs

    tabs = load_tabs(0, 0)

    def on_file_change(event):
        if fileInput.filename is None: return

        try:
            lista = fileInput.name.split("/")
            active_campaign = lista[0]
            active_stage = lista[1]
        except:
            active_campaign = tabs.objects[0].name
            active_stage = tabs.objects[0][0].name          

        fileName = fileInput.filename
        data = fileInput.value

        path = f"/content/drive/MyDrive/{PROJECT_NAME}/{active_campaign}/{active_stage}"
        if not os.path.exists(path):
            os.makedirs(path)

        path = f"/content/drive/MyDrive/{PROJECT_NAME}/{active_campaign}/{active_stage}/{fileName}"
        with open(path, "wb") as f:
            f.write(data)

        endpoint_addDocument(active_campaign, active_stage, path) 
        try:
          lista = fileInput.message.split("/")
          layout[0] = load_tabs(int(lista[0]), int(lista[1]))  
        except:
          layout[0] = load_tabs(0, 0)

        fileInput.value = None
        fileInput.filename = None

    fileInput.param.watch(on_file_change, 'filename')
    layout = panel.Row(tabs, name='Bases de Conhecimento, Estágios e Arquivos')
    return layout


print("\nGUI de Adição/Exclusão de Arquivos preparada!\n")
#display(gui_CampaignFiles(None))


GUI de Adição/Exclusão de Arquivos preparada!



In [24]:
# GUI Config ChromaDB
def gui_chromadb(refresh_callback):

    margin = 20

    chunks_input = panel.widgets.IntInput(value=750, step=1)
    overlap_input = panel.widgets.IntInput(value=80, step=1)
    select_embeddings_type = panel.widgets.RadioButtonGroup(
        name='Embeddings type', 
        options=['OpenAIEmbeddings','HuggingFaceEmbeddings'], 
        button_type='primary'
    )
    btn = panel.widgets.Button(name="Reconstruir Base de Dados", button_type='primary')

    campaigns = endpoint_getCampaigns()
    def loadOptions(items):
        opts = []
        for key, tmp in items: opts.append(key) 
        return opts

    select_campaign = panel.widgets.RadioButtonGroup(button_type='primary') 
    select_campaign.options = loadOptions(campaigns.items())
    os.environ['ACTIVE_CAMPAIGN'] = select_campaign.options[0]
    select_campaign.value = select_campaign.options[0]


    def save_conf(key, value):
        campaign = select_campaign.value
        os.environ['ACTIVE_CAMPAIGN'] = campaign
        campaigns = endpoint_getCampaigns()
        confs = campaigns[campaign]["confs"]
        confs[key] = value
        endpoint_setCampaigns(campaigns)

    def update_max_chunks(event): save_conf("chunk_size", event.new)
    def update_overlap(event): save_conf("chunk_overlap", event.new)
    def update_embedding(event): 
        save_conf("embeddings", event.new)

    select_embeddings_type.param.watch(update_embedding, 'value')
    chunks_input.param.watch(update_max_chunks, 'value')
    overlap_input.param.watch(update_overlap, 'value')

    def loadParameters(campaign_name):
        campaigns = endpoint_getCampaigns()
        confs = campaigns[campaign_name]["confs"]
        chunks_input.value = confs["chunk_size"]
        overlap_input.value = confs["chunk_overlap"]
        select_embeddings_type.value = confs["embeddings"]
        os.environ['ACTIVE_CAMPAIGN'] = campaign_name

    loadParameters(select_campaign.value)

    def buid_gui():
        return panel.Row(
            panel.Column("Tipo de Embedding:", select_embeddings_type, width=350),
            panel.Column("Max Chunks:", chunks_input, width=120),   
            panel.Column("Overlap:", overlap_input, width=120),  
            panel.Column(" . ", btn),
            margin=margin, background=LIGHT_GRAY
        )

    def update_campaign(event):
        clear_output(wait=True)
        display(panel.Row())
        loadParameters(select_campaign.value)
        refresh_callback()

    select_campaign.param.watch(update_campaign, 'value')

    def clearLog(event):
        refresh_callback()

    clear = panel.widgets.Button(name="Limpar Log")
    clear.on_click(clearLog)

    result = panel.Column( 
        panel.Row(select_campaign, margin=margin),
        buid_gui(),
        panel.Row(clear, width=950, margin=margin), 
        margin=margin, 
        width=950, 
        name="Base de Dados Semântica"
    )
    
    def buildSemanticDB(event):
      loadParameters(select_campaign.value)
      logger_manager.setupLogger(logging.INFO)
      name = select_campaign.value
      buildVectorStore(name)
      logger_manager.setupLogger(logging.ERROR)
      refresh_callback()

    btn.on_click(buildSemanticDB)
    return result

print("\nGUI de Configuração Semantica preparada!\n")


[ERROR] [base_events] [default_exception_handler @1758] -> Future exception was never retrieved
future: <Future finished exception=TypeError("download() got an unexpected keyword argument 'format'")>
Traceback (most recent call last):
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 128, in _extractFromHtmlWithGDownAndSoup
    filePath = self._downloadGDriveFileToTemp(gFileId, "txt")
  File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 94, in _downloadGDriveFileToTemp
    path = gdown.download(id = id, output = path, quiet = False, format = exportFormat, use_cookies = False)
TypeError: download() got an unexpected keyword argument 'format'
[ERROR] [base_events] [default_exception_handler @1758] -> Future exception was never retrieved
future: <Future finished exception=TypeError("download() got an

In [25]:
# GUI Answering Config
panel.extension()
def gui_answeringConfig(refresh_callback):

    margin=20

    select_campaign = panel.widgets.RadioButtonGroup(button_type='primary') 
    select_stage = panel.widgets.RadioButtonGroup(button_type='primary') 
    
    campaigns = endpoint_getCampaigns()
    def loadOptions(items):
        opts = []
        for key, tmp in items: opts.append(key) 
        return opts

    select_campaign.options = loadOptions(campaigns.items())
    os.environ['ACTIVE_CAMPAIGN'] = select_campaign.options[0]
    select_campaign.value = select_campaign.options[0]

    select_stage.options = loadOptions(campaigns[select_campaign.options[0]]["stages"].items())
    os.environ['ACTIVE_STAGE'] = select_stage.options[0]
    select_stage.value = select_stage.options[0]

    select_chain_type = panel.widgets.RadioButtonGroup(
        options=['stuff', 'map_reduce', "refine", "map_rerank"], button_type='primary'
    )  
    os.environ['CHAIN_TYPE'] = select_chain_type.options[0]
    select_chain_type.value = select_chain_type.options[0]

    select_search_type = panel.widgets.RadioButtonGroup(
        options=["similarity", "mmr", "hybrid"], button_type='primary'
    ) 
    os.environ['SEARCH_TYPE'] = select_search_type.options[0]
    select_search_type.value = select_search_type.options[0]

    select_chat_type = panel.widgets.RadioButtonGroup(
        options=["QNA", "Prompt"], button_type='primary'
    ) 
    os.environ['CHAT_TYPE'] = select_chat_type.options[0]
    select_chat_type.value = select_chat_type.options[0]

    select_k = panel.widgets.IntInput(start=1, end=10, step=1, value=5)  
    os.environ['RELEVANT_CHUNKS'] = f"{select_k.value}"

    global textarea

    def update_chain_type(event): os.environ['CHAIN_TYPE'] = select_chain_type.value
    def update_relevant_chunks(event): os.environ['RELEVANT_CHUNKS'] = f"{select_k.value}"  
    def update_search_type(event): os.environ['SEARCH_TYPE'] = select_search_type.value
    def update_chat_type(event): os.environ['CHAT_TYPE'] = select_chat_type.value
    def update_stage(event): ACTIVE_STAGE = select_stage.value

    def update_campaign(event): 
        campaigns = endpoint_getCampaigns()
        os.environ['ACTIVE_CAMPAIGN'] = select_campaign.value

        opts = loadOptions(campaigns[select_campaign.value]["stages"].items())
        os.environ['ACTIVE_STAGE'] = opts[0]
        select_stage.options = opts
        select_stage.value = opts[0]
        refresh_callback()
        print()

    select_chain_type.param.watch(update_chain_type, 'value')
    select_search_type.param.watch(update_search_type, 'value')
    select_k.param.watch(update_relevant_chunks, 'value')
    select_chat_type.param.watch(update_chat_type, 'value')
    select_campaign.param.watch(update_campaign, 'value')
    select_stage.param.watch(update_stage, 'value')

    def clearLog(event):
        refresh_callback()

    btn = panel.widgets.Button(name="Limpar Log")
    btn.on_click(clearLog)


    return panel.Column(
        panel.Row(panel.Column("Campanha:", select_campaign), margin=margin),
        panel.Row(panel.Column("Estágio:", select_stage), margin=margin),
        panel.Row(
            panel.Column("Chat Type:", select_chat_type, width=200),
            panel.Column("Tipo da Chain:", select_chain_type, width=300),
            panel.Column("Search Type:", select_search_type, width=200),
            panel.Column("Chunks Relevantes:", select_k, width=150),
            margin=margin    
        ),  
        btn, name="Chatting Configuration", margin=margin
    )
    
print("\nGUI Answering Config preparada!\n")


GUI Answering Config preparada!



In [26]:
# GUI Principal
panel.extension()
def gui_Main(refresh_callback):
    layout = panel.Accordion(width=1000, background=LIGHT_GRAY)
    layout.append(gui_CampaignFiles(refresh_callback))
    layout.append(gui_chromadb(refresh_callback))
    layout.append(gui_answeringConfig(refresh_callback))
    layout.active = [0]
    #layout.min_width = 0

    def toggle_active(event):
        #if len(layout.active)<=1: return        
        #toRemove = layout.min_width
        #for i in reversed(range(len(layout.active))):
        #    if layout.active[i] == toRemove: 
        #        layout.active.pop(i)
        #        layout.objects[i].active=False
        #        continue
         #   layout.min_width = layout.active[i]
        refresh_callback()

    layout.param.watch(toggle_active, 'active', onlychanged=True)
    return layout


def startApp():
    refs = []

    def refreshDisplay():
        clear_output(wait=True)
        display(refs[0]) 

    refs.append(gui_Main(refreshDisplay))
    display(refs[0])


print("\nGUI principal preparada!\n")
#display(gui_Main(None))


GUI principal preparada!



## AI


### Training and Config

In [27]:
# Abrir GUI
startApp()

Accordion(active=[1], background='#FFFFFF', width=1000)
    [0] Row(name='Bases de Conhecimento, ...)
        [0] Tabs(background='#FFFFFF', margin=20)
            [0] Tabs(margin=20, name='Catavento')
                [0] Column(background='#FFFFFF', name='RH')
                    [0] Column
                        [0] Tabulator(buttons={'delete': "<img s...}, disabled=True, hidden_columns=['Campanha', 'Estágio', ...], show_index=False, value=    Campanha E..., width=900, widths={'Arquivo': 858})
                    [1] Column(background='#FFFFFF', width=930)
                        [0] Row
                            [0] Markdown(str)
                            [1] FileInput(margin=10)
    [1] Column(margin=20, name='Base de Dados Semântica', width=950)
        [0] Row(margin=20)
            [0] RadioButtonGroup(button_type='primary', options=['Catavento'], value='Catavento')
        [1] Row(background='#FFFFFF', margin=20)
            [0] Column(width=350)
                [0] Markdown(str)
                [1] RadioButtonGroup(button_type='primary', name='Embeddings type', options=['OpenAIEmbeddings', ...], value='OpenAIEmbeddings')
            [1] Column(width=120)
                [0] Markdown(str)
                [1] IntInput(value=750)
            [2] Column(width=120)
                [0] Markdown(str)
                [1] IntInput(value=80)
            [3] Column
                [0] Markdown(str)
                [1] Button(button_type='primary', clicks=1, name='Reconstruir B...)
        [2] Row(margin=20, width=950)
            [0] Button(name='Limpar Log')
    [2] Column(margin=20, name='Chatting Configuration')
        [0] Row(margin=20)
            [0] Column
                [0] Markdown(str)
                [1] RadioButtonGroup(button_type='primary', options=['Catavento'], value='Catavento')
        [1] Row(margin=20)
            [0] Column
                [0] Markdown(str)
                [1] RadioButtonGroup(button_type='primary', options=['RH'], value='RH')
        [2] Row(margin=20)
            [0] Column(width=200)
                [0] Markdown(str)
                [1] RadioButtonGroup(button_type='primary', options=['QNA', 'Prompt'], value='QNA')
            [1] Column(width=300)
                [0] Markdown(str)
                [1] RadioButtonGroup(button_type='primary', options=['stuff', 'map_reduce', ...], value='stuff')
            [2] Column(width=200)
                [0] Markdown(str)
                [1] RadioButtonGroup(button_type='primary', options=['similarity', ...], value='similarity')
            [3] Column(width=150)
                [0] Markdown(str)
                [1] IntInput(end=10, start=1, value=5)
        [3] Button(name='Limpar Log')

Catavento/RH: 750 | 80 
[INFO] [logger_manager] [info @27] -> document '3a9cd09df979aa57c09722992b181502' generating raw content chunks, chunk size '750', chunk overlap '80' 
[INFO] [SentenceTransformer] [__init__ @66] -> Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2 
[INFO] [SentenceTransformer] [__init__ @105] -> Use pytorch device: cpu 
[INFO] [posthog] [__init__ @15] -> Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information. 
[INFO] [__init__] [Client @80] -> Running Chroma using direct local API. 
[WARNING] [__init__] [get_db @41] -> Using embedded DuckDB with persistence: data will be stored in: /content/drive/MyDrive/colab.ai/Catavento/RH/chromaDB/chroma_vector_store_data 
[INFO] [ctypes] [ @22] -> Successfully imported ClickHouse Connect C data optimizations 
[INFO] [ctypes] [ @31] -> Successfully import ClickHouse Connect C/Numpy optimizations 
[INFO] [json_impl] [set_json_library @45] -> Using python library for writing JSON byte strings 
[ERROR] [base_events] [default_exception_handler @1758] -> Future exception was never retrieved 
future: 
Traceback (most recent call last): 
 File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run 
 result = self.fn(*self.args, **self.kwargs) 
 File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 128, in _extractFromHtmlWithGDownAndSoup 
 filePath = self._downloadGDriveFileToTemp(gFileId, "txt") 
 File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 94, in _downloadGDriveFileToTemp 
 path = gdown.download(id = id, output = path, quiet = False, format = exportFormat, use_cookies = False) 
TypeError: download() got an unexpected keyword argument 'format' 
[ERROR] [base_events] [default_exception_handler @1758] -> Future exception was never retrieved 
future: 
Traceback (most recent call last): 
 File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run 
 result = self.fn(*self.args, **self.kwargs) 
 File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 166, in _extractFromPdfWithGDownAndFitz 
 filePath = self._downloadGDriveFileToTemp(gFileId, None) 
 File "/content/git/jai-bot-server/src/apis/documents_wizard/url_extractor.py", line 94, in _downloadGDriveFileToTemp 
 path = gdown.download(id = id, output = path, quiet = False, format = exportFormat, use_cookies = False) 
TypeError: download() got an unexpected keyword argument 'format' 
[INFO] [duckdb] [load @426] -> No existing DB found in /content/drive/MyDrive/colab.ai/Catavento/RH/chromaDB/chroma_vector_store_data, skipping load 
[INFO] [duckdb] [load @436] -> No existing DB found in /content/drive/MyDrive/colab.ai/Catavento/RH/chromaDB/chroma_vector_store_data, skipping load 
[INFO] [logger_manager] [info @27] -> adding '98' chunks of raw content on vector store for document '3a9cd09df979aa57c09722992b181502' 
[INFO] [logger_manager] [info @27] -> adding '0' chunks of improved content on vector store for document '3a9cd09df979aa57c09722992b181502' 
[INFO] [duckdb] [persist @388] -> Persisting DB to disk, putting it in the save folder: /content/drive/MyDrive/colab.ai/Catavento/RH/chromaDB/chroma_vector_store_data 
[INFO] [duckdb] [__del__ @445] -> PersistentDuckDB del, about to run persist 
[INFO] [duckdb] [persist @388] -> Persisting DB to disk, putting it in the save folder: /content/drive/MyDrive/colab.ai/Catavento/RH/chromaDB/chroma_vector_store_data

### Answering

In [11]:
# Pergunte Alguma Coisa
log = []
do_chat()


None | None


TypeError: ignored